In [2]:
import os
import time
import cv2

#load_data
def load_data(path):
    X = []
    y = []
    i=0
    for dirpath,dirsname,filesname in os.walk(path,topdown=True):
        for filename in filesname:
            filename_path = os.sep.join([dirpath,filename])
            X.append(filename_path)
            y.append(i-1)
        i+=1
    return X,y
X,y = load_data("../DataSet/cnn_char_train")

In [3]:
data = []
for path in X:
    data.append(cv2.imread(path))

len(data)

16148

In [4]:
print(data[0].shape)

(20, 20, 3)


In [11]:
# part_card = cv2.copyMakeBorder(part_card, 0, 0, w, w, cv2.BORDER_CONSTANT, value = [0,0,0])
# part_card = cv2.resize(part_card, (SZ, SZ), interpolation=cv2.INTER_AREA)

# #part_card = deskew(part_card)
# part_card = preprocess_hog([part_card])
WIN_SIZE = (8,8)
BLOCK_SIZE = (4,4)
BLOCK_STRIDE = (4,4)
CELL_SZIE = (2,2)
NBINS = 4
data_features = []
for x in data:
    #x = cv2.copyMakeBorder(x, 0, 0, 10, 10, cv2.BORDER_CONSTANT, value = [0,0,0])
    #x = cv2.resize(x, (20, 20), interpolation=cv2.INTER_AREA)
    hog = cv2.HOGDescriptor(WIN_SIZE,BLOCK_SIZE,BLOCK_STRIDE,CELL_SZIE,NBINS)
    x_features = hog.compute(x)
    data_features.append(x_features)
data_features[0][0]

array([0.00866267], dtype=float32)

In [12]:
import numpy as np
X = np.array(data_features).reshape(16148,3136)
print(X.shape)

(16148, 3136, 1)


In [48]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
clf = svm.SVC(kernel="rbf",gamma=5,C=0.001)
scores = cross_val_score(clf,X,y,cv=5)
scores

ValueError: Found array with dim 3. Estimator expected <= 2.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torch.optim as optim

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 输入图像channel：1；输出channel：6；5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 2x2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除去批处理维度的其他所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
#数据加载
class DanTongDao(object):
    def __call__(self, pics):
        pics = np.array(pics)
        #pics = cv2.resize(pics, (20, 20), interpolation=cv2.INTER_AREA)
        pics = cv2.cvtColor(pics,cv2.COLOR_BGR2GRAY)
        #pics = cv2.copyMakeBorder(pics, 0, 0, 5, 5, cv2.BORDER_CONSTANT, value = [0,0,0])
        PIL_image = Image.fromarray(pics) 
        return PIL_image
    def __repr__(self):
        return self.__class__.__name__ + '()'
data_transform = transforms.Compose([
        DanTongDao(),
        transforms.CenterCrop((20,20)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5,],
                             std=[0.5,])
    ])
hymenoptera_dataset = datasets.ImageFolder(root='../DataSet/cnn_char_train/',
                                           transform=data_transform)
print(len(hymenoptera_dataset))
dataset_loader = torch.utils.data.DataLoader(hymenoptera_dataset,
                                             batch_size=8, shuffle=True,
                                             num_workers=0)

16148


In [7]:
# 损失函数
loss = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)


In [ ]:
for epoch in range(15):
    running_loss = 0.0
    for i,data in enumerate(datset_loader):
        X,y = data
        optimizer.zero_grad()
        out = net(x)
        